In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


### Storage Parameters

In [ ]:
Ts = 1/12
c = 20
eta = 0.9
case_template = 'Strategic/MC20/UC25ED1_Strategic_true_ratio{}_Seg1_BAW0_MC20.0'
ratios = [round(i * 0.1, 1) for i in range(11)]
# case_template = 'Strategic/EDH6/UC25ED6_Strategic_true_ratio1.0_Seg1_BAW0_MC20.0_{}'
# ratios = range(10)

# case = 'Strategic/MC20/Strategic/UC25ED1_Strategic_true_ratio0.3_Seg1_BAW0_MC20.0'
# case = 'Strategic/UC25ED1_Strategic_true_Seg1_Load1.0_Fuel1.2_Error0.25_ratio1.0_MIP0.1_DARTDP_Hete'
# case = 'Strategic/BidAhead/UC25ED1_Strategic_true_Seg1_Load1.0_Fuel1.2_Error0.25_ratio1.0_MIP0.1_DARTDP_BAW36'

### Price Stats

In [ ]:
# Initialize lists to hold DataFrames
mean_prices_dfs = []
std_prices_dfs = []

In [ ]:
# Loop over ratios from 0.0 to 1.0 (assuming increment by 0.1)
for ratio in ratios:
    case = case_template.format(ratio)
    try:
        # Try to read the CSV file for the current case
        price = pd.read_csv('output/'+case+'/EDprice.csv', header=None)
    except FileNotFoundError:
        print(f"File for case {case} not found.")
        continue

    # Calculate stats and transpose
    pricestats = pd.concat([price.mean()*12, (price*12).std()], axis=1).T

    # Now, instead of appending Series objects, you create DataFrame slices for each and append them
    mean_prices_dfs.append(pricestats.iloc[[0]])
    std_prices_dfs.append(pricestats.iloc[[1]])

In [ ]:
# Convert lists of series to DataFrames
mean_prices_df = pd.concat(mean_prices_dfs).reset_index(drop=True)
std_prices_df = pd.concat(std_prices_dfs).reset_index(drop=True)
mean_prices_df.index = [str(round(i * 0.1, 1)) for i in range(len(mean_prices_dfs))]
std_prices_df.index = [str(round(i * 0.1, 1)) for i in range(len(std_prices_dfs))]

In [ ]:
mean_prices_df

In [ ]:
std_prices_df

In [ ]:
# Selecting the columns for Regions 1, 2, and 3
regions = [0, 1, 2]
mean_prices = mean_prices_df.iloc[:, regions]
std_prices = std_prices_df.iloc[:, regions]

# Preparing ISR values from the index
isr_values = mean_prices.index

# New colors and error bar theme
new_colors = ['#0072B2', '#D55E00', '#CC79A7']  # Updated color palette
new_labels = ['Region 1', 'Region 2', 'Region 3']
line_styles = ['-', '--', '-.']  # Different line styles for visual distinction
marker_styles = ['o', 's', 'd']  # Different marker styles for each region
plt.figure(figsize=(8, 6))


for region, color, label, line_style, marker_style in zip(regions, new_colors, new_labels, line_styles, marker_styles):
    plt.errorbar(isr_values, mean_prices.iloc[:, region], yerr=std_prices.iloc[:, region], 
                 label=label, color=color, fmt=line_style + marker_style, capsize=5, elinewidth=2, capthick=2, markeredgewidth=2)

plt.title('CAISO Real-Time Price Variability by Region and ISR')
plt.xlabel('Intelligent Storage Ratio (ISR)')
plt.ylabel('Mean Price ($/MWh)')
plt.ylim([-20,190])
plt.legend()
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.tight_layout()


# Save the plot to a file suitable for inclusion in your paper
# plt.savefig('/mnt/data/caiso_price_variability.png', dpi=300)

# Show the plot
plt.show()


### Total System Cost

In [ ]:
total_system_costs = []
# Loop over ratios from 0.0 to 1.0 (assuming increment by 0.1)
for ratio in ratios:
    case = case_template.format(ratio)
    try:
        # Read the CSV file for the current case
        cost = pd.read_csv(f'output/{case}/EDcost.csv', header=None)
    except FileNotFoundError:
        print(f"File for case {case} not found.")
        # Optionally, append a NaN or some indicator of missing data
        total_system_costs.append(float('nan'))
        continue

    # Sum the total cost and normalize
    total_cost_normalized = cost.sum().values[0] / (10**10)
    total_system_costs.append(total_cost_normalized)

# Convert the list to a DataFrame
total_system_costs_df = pd.DataFrame(total_system_costs, columns=['Total System Cost'])

# Adjust the index to represent ratios directly, if desired
# total_system_costs_df.index = [str(round(i * 0.1, 1)) for i in range(11)]


total_system_costs_df

In [ ]:
ratios

In [ ]:
# Calculate percentage reduction compared to 0.0 ISR
initial_cost = total_system_costs_df['Total System Cost'].iloc[0]
percentage_reduction = (total_system_costs_df['Total System Cost'] - initial_cost) / initial_cost * 100

# Plotting
fig, ax1 = plt.subplots(figsize=(8, 6))

color = '#0072B2'
ax1.set_xlabel('Intelligent Storage Ratio (ISR)', fontsize=12)
ax1.set_ylabel('Total System Cost (Billion $)', color=color, fontsize=12)
ax1.plot(ratios, total_system_costs_df['Total System Cost'], '-o', color=color, label='Total System Cost')
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(True, which='both', linestyle='--', linewidth=0.5)

# Instantiate a second y-axis for percentage reduction
ax2 = ax1.twinx() 
color = '#D55E00'
ax2.set_ylabel('Percentage Reduction (%)', color=color, fontsize=12)  
ax2.bar(ratios, percentage_reduction, width=0.05, color=color, alpha=0.6, label='Percentage Reduction')
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  
plt.title('Total System Cost and Percentage Reduction Across WECC vs ISR', fontsize=14)
plt.show()


### Storage Profit & Utilization

In [ ]:
# case = 'Strategic/MC20/UC25ED1_Strategic_true_ratio0.9_Seg5_BAW0_MC20.0'
case = 'Strategic/EDH6/UC25ED6_Strategic_true_ratio1.0_Seg1_BAW0_MC20.0_1'


ESD = pd.read_csv('output/'+case+'/EDESD.csv',header=None)
ESC = pd.read_csv('output/'+case+'/EDESC.csv',header=None)
price = pd.read_csv('output/'+case+'/EDprice.csv',header=None)
ES = pd.read_csv('output/'+case+'/Strategic/ADDED_ES.csv')

In [ ]:
data = [
    ['S1', sum((ESD[35]-ESC[35])*price[0])/1000, sum((ESD[35]-ESC[35])*price[0] - Ts*c*ESD[35])/1000, sum((ESD[35]-ESC[35])*price[0] - Ts*c*ESD[35])/1000/ES['ESOC'][0], sum(ESD[35])/eta*Ts/ES['ESOC'][0]],
    ['NS1', sum((ESD[36]-ESC[36])*price[0])/1000, sum((ESD[36]-ESC[36])*price[0] - Ts*c*ESD[36])/1000, sum((ESD[36]-ESC[36])*price[0] - Ts*c*ESD[36])/1000/ES['ESOC'][1], sum(ESD[36])/eta*Ts/ES['ESOC'][1]],
    ['S2', sum((ESD[37]-ESC[37])*price[1])/1000, sum((ESD[37]-ESC[37])*price[1] - Ts*c*ESD[37])/1000, sum((ESD[37]-ESC[37])*price[1] - Ts*c*ESD[37])/1000/ES['ESOC'][2], sum(ESD[37])/eta*Ts/ES['ESOC'][2]],
    ['NS2', sum((ESD[38]-ESC[38])*price[1])/1000, sum((ESD[38]-ESC[38])*price[1] - Ts*c*ESD[38])/1000, sum((ESD[38]-ESC[38])*price[1] - Ts*c*ESD[38])/1000/ES['ESOC'][3], sum(ESD[38])/eta*Ts/ES['ESOC'][3]],
    ['S3', sum((ESD[39]-ESC[39])*price[2])/1000, sum((ESD[39]-ESC[39])*price[2] - Ts*c*ESD[39])/1000, sum((ESD[39]-ESC[39])*price[2] - Ts*c*ESD[39])/1000/ES['ESOC'][4], sum(ESD[39])/eta*Ts/ES['ESOC'][4]],
    ['NS3', sum((ESD[40]-ESC[40])*price[2])/1000, sum((ESD[40]-ESC[40])*price[2] - Ts*c*ESD[40])/1000, sum((ESD[40]-ESC[40])*price[2] - Ts*c*ESD[40])/1000/ES['ESOC'][5], sum(ESD[40])/eta*Ts/ES['ESOC'][5]],
    ['S4', sum((ESD[41]-ESC[41])*price[3])/1000, sum((ESD[41]-ESC[41])*price[3] - Ts*c*ESD[41])/1000, sum((ESD[41]-ESC[41])*price[3] - Ts*c*ESD[41])/1000/ES['ESOC'][6], sum(ESD[41])/eta*Ts/ES['ESOC'][6]],
    ['NS4', sum((ESD[42]-ESC[42])*price[3])/1000, sum((ESD[42]-ESC[42])*price[3] - Ts*c*ESD[42])/1000, sum((ESD[42]-ESC[42])*price[3] - Ts*c*ESD[42])/1000/ES['ESOC'][7], sum(ESD[42])/eta*Ts/ES['ESOC'][7]],
    ['S5', sum((ESD[43]-ESC[43])*price[4])/1000, sum((ESD[43]-ESC[43])*price[4] - Ts*c*ESD[43])/1000, sum((ESD[43]-ESC[43])*price[4] - Ts*c*ESD[43])/1000/ES['ESOC'][8], sum(ESD[43])/eta*Ts/ES['ESOC'][8]],
    ['NS5', sum((ESD[44]-ESC[44])*price[4])/1000, sum((ESD[44]-ESC[44])*price[4] - Ts*c*ESD[44])/1000, sum((ESD[44]-ESC[44])*price[4] - Ts*c*ESD[44])/1000/ES['ESOC'][9], sum(ESD[44])/eta*Ts/ES['ESOC'][9]],
    ['S6', sum((ESD[45]-ESC[45])*price[5])/1000, sum((ESD[45]-ESC[45])*price[5] - Ts*c*ESD[45])/1000, sum((ESD[45]-ESC[45])*price[5] - Ts*c*ESD[45])/1000/ES['ESOC'][10], sum(ESD[45])/eta*Ts/ES['ESOC'][10]],
    ['NS6', sum((ESD[46]-ESC[46])*price[5])/1000, sum((ESD[46]-ESC[46])*price[5] - Ts*c*ESD[46])/1000, sum((ESD[45]-ESC[45])*price[5] - Ts*c*ESD[46])/1000/ES['ESOC'][11], sum(ESD[46])/eta*Ts/ES['ESOC'][11]],
]
column_names = ['', 'Revenue (k$)', 'Profit (k$)', 'Unit Profit (k$/MWh)', 'Cycle']
df = pd.DataFrame(data, columns=column_names)
df.T


In [ ]:
ESD1 = pd.read_csv('output/Strategic/UC25ED1_Strategic_true_Seg1_Load1.0_Fuel1.2_Error0.25_ratio0.0001_MIP0.1_DARTDP/EDESD.csv',header=None)
ESC1 = pd.read_csv('output/Strategic/UC25ED1_Strategic_true_Seg1_Load1.0_Fuel1.2_Error0.25_ratio0.0001_MIP0.1_DARTDP/EDESC.csv',header=None)
price1 = pd.read_csv('output/Strategic/UC25ED1_Strategic_true_Seg1_Load1.0_Fuel1.2_Error0.25_ratio0.0001_MIP0.1_DARTDP/EDprice.csv',header=None)
DAprice1 = pd.read_csv('output/Strategic/UC25ED1_Strategic_true_Seg1_Load1.0_Fuel1.2_Error0.25_ratio0.5_MIP0.1_DARTDP/UCprice.csv',header=None)

### Old Storage Profit

In [ ]:
repeated_indices = np.repeat(DAprice1.index, 12)
repeated_DAprice1 = DAprice1.loc[repeated_indices].reset_index(drop=True)

In [ ]:
plt.figure(figsize=(10, 6))  # Optional: Specifies the figure size
plt.plot(range(2017), price1.loc[0:2016,0]*12, label='RT')  # Plot first column
plt.plot(range(2017), repeated_DAprice1.loc[0:2016,0], label='DA', linestyle='--')  # Plot second column with a different style

# Adding labels and title
# plt.xlabel('Index')  # Adjust as per your requirement
plt.ylabel('$/MWh')  # Adjust as per your requirement
# plt.title('Line Plot of Two DataFrame Columns')
plt.legend()

# Show plot
plt.show()

In [ ]:
ESD1 = pd.read_csv('output/Strategic/UC25ED1_Strategic_true_Seg1_Load1.0_Fuel1.2_Error0.25_ratio0.5_MIP0.1_DARTDP/EDESD.csv',header=None)
ESC1 = pd.read_csv('output/Strategic/UC25ED1_Strategic_true_Seg1_Load1.0_Fuel1.2_Error0.25_ratio0.5_MIP0.1_DARTDP/EDESC.csv',header=None)
price1 = pd.read_csv('output/Strategic/UC25ED1_Strategic_true_Seg1_Load1.0_Fuel1.2_Error0.25_ratio0.5_MIP0.1_DARTDP/EDprice.csv',header=None)

ESD5 = pd.read_csv('output/Strategic/UC25ED1_Strategic_true_Seg5_Load1.0_Fuel1.2_Error0.25_ratio0.5_MIP0.1_DARTDP/EDESD.csv',header=None)
ESC5 = pd.read_csv('output/Strategic/UC25ED1_Strategic_true_Seg5_Load1.0_Fuel1.2_Error0.25_ratio0.5_MIP0.1_DARTDP/EDESC.csv',header=None)
price5 = pd.read_csv('output/Strategic/UC25ED1_Strategic_true_Seg5_Load1.0_Fuel1.2_Error0.25_ratio0.5_MIP0.1_DARTDP/EDprice.csv',header=None)

ESD1mar = pd.read_csv('output/Strategic/UC25ED1_Strategic_true_Seg1_Load1.0_Fuel1.2_Error0.25_ratio0.0001_MIP0.1_DARTDP/EDESD.csv',header=None)
ESC1mar = pd.read_csv('output/Strategic/UC25ED1_Strategic_true_Seg1_Load1.0_Fuel1.2_Error0.25_ratio0.0001_MIP0.1_DARTDP/EDESC.csv',header=None)
price1mar = pd.read_csv('output/Strategic/UC25ED1_Strategic_true_Seg1_Load1.0_Fuel1.2_Error0.25_ratio0.0001_MIP0.1_DARTDP/EDprice.csv',header=None)

ESD5mar = pd.read_csv('output/Strategic/UC25ED1_Strategic_true_Seg5_Load1.0_Fuel1.2_Error0.25_ratio0.0001_MIP0.1_DARTDP/EDESD.csv',header=None)
ESC5mar = pd.read_csv('output/Strategic/UC25ED1_Strategic_true_Seg5_Load1.0_Fuel1.2_Error0.25_ratio0.0001_MIP0.1_DARTDP/EDESC.csv',header=None)
price5mar = pd.read_csv('output/Strategic/UC25ED1_Strategic_true_Seg5_Load1.0_Fuel1.2_Error0.25_ratio0.0001_MIP0.1_DARTDP/EDprice.csv',header=None)


In [ ]:
Zone2Price1 = price1[0]
SD1 = ESD1[35]
SC1 = ESC1[35]
NSD1 = ESD1[36]
NSC1 = ESC1[36]
Zone2Price5 = price5[0]
SD5 = ESD5[35]
SC5 = ESC5[35]
NSD5 = ESD5[36]
NSC5 = ESC5[36]
Zone2Price1mar = price1mar[0]
SD1mar = ESD1mar[35]
SC1mar = ESC1mar[35]
NSD1mar = ESD1mar[36]
NSC1mar = ESC1mar[36]
Zone2Price5mar = price5mar[1]
SD5mar = ESD5mar[35]
SC5mar = ESC5mar[35]
NSD5mar = ESD5mar[36]
NSC5mar = ESC5mar[36]

In [ ]:
S1 = SD1-SC1
NS1 = NSD1-NSC1
S5 = SD5-SC5
NS5 = NSD5-NSC5
S1mar = SD1mar-SC1mar
NS1mar = NSD1mar-NSC1mar
S5mar = SD5mar-SC5mar
NS5mar = NSD5mar-NSC5mar

In [ ]:
Revenue_S1 = sum(S1*Zone2Price1) 
Profit_S1 = sum(S1*Zone2Price1 - Ts*c*SD1)
Cycle_S1 = sum(SD1)/eta*Ts/E
Revenue_NS1 = sum(NS1*Zone2Price1)
Profit_NS1 = sum(NS1*Zone2Price1 - Ts*c*NSD1)
Cycle_NS1 = sum(NSD1)/eta*Ts/E


In [ ]:
Revenue_S5 = sum(S5*Zone2Price5) 
Profit_S5 = sum(S5*Zone2Price5 - Ts*c*SD5)
Cycle_S5 = sum(SD5)/eta*Ts/E
Revenue_NS5 = sum(NS5*Zone2Price5)
Profit_NS5 = sum(NS5*Zone2Price5 - Ts*c*NSD5) 
Cycle_NS5 = sum(NSD5)/eta*Ts/E


In [ ]:
Revenue_S1mar = sum(S1mar*Zone2Price1mar) 
Profit_S1mar = sum(S1mar*Zone2Price1mar - Ts*c*SD1mar)
Cycle_S1mar = sum(SD1mar)/eta*Ts/E1
Revenue_NS1mar = sum(NS1mar*Zone2Price1mar)
Profit_NS1mar = sum(NS1mar*Zone2Price1mar - Ts*c*NSD1mar) 
Cycle_NS1mar = sum(NSD1mar)/eta*Ts/E2

In [ ]:
Revenue_S5mar = sum(S5mar*Zone2Price5mar) 
Profit_S5mar = sum(S5mar*Zone2Price5mar - Ts*c*SD5mar)
Cycle_S5mar = sum(SD5mar)/eta*Ts/E1
Revenue_NS5mar = sum(NS5mar*Zone2Price5mar)
Profit_NS5mar = sum(NS5mar*Zone2Price5mar - Ts*c*NSD5mar) 
Cycle_NS5mar = sum(NSD5mar)/eta*Ts/E2

In [ ]:
data = [
    ['S-1Seg', Revenue_S1/1000, Profit_S1/1000, Profit_S1/1000/E, Cycle_S1, Profit_S1/1000/E/27.992],
    ['NS-1Seg', Revenue_NS1/1000, Profit_NS1/1000, Profit_NS1/1000/E, Cycle_NS1, Profit_NS1/1000/E/27.992],
    ['S-5Seg', Revenue_S5/1000, Profit_S5/1000, Profit_S5/1000/E, Cycle_S5, Profit_S5/1000/E/27.992],
    ['NS-5Seg', Revenue_NS5/1000, Profit_NS5/1000, Profit_NS5/1000/E, Cycle_NS5, Profit_NS5/1000/E/27.992],
    ['S-1Seg_mar', Revenue_S1mar/1000, Profit_S1mar/1000, Profit_S1mar/1000/E1, Cycle_S1mar, Profit_S1mar/1000/E1/27.992],
    ['NS-1Seg_mar', Revenue_NS1mar/1000, Profit_NS1mar/1000, Profit_NS1mar/1000/E2, Cycle_NS1mar, Profit_NS1mar/1000/E2/27.992],
    ['S-5Seg_mar', Revenue_S5mar/1000, Profit_S5mar/1000, Profit_S5mar/1000/E1, Cycle_S5mar, Profit_S5mar/1000/E1/27.992],
    ['NS-5Seg_mar', Revenue_NS5mar/1000, Profit_NS5mar/1000, Profit_NS5mar/1000/E2, Cycle_NS5mar, Profit_NS5mar/1000/E2/27.992],
]
column_names = ['', 'Revenue (k$)', 'Profit (k$)', 'Unit Profit (k$/MWh)', 'Cycle', 'Profit Ratio']
df = pd.DataFrame(data, columns=column_names)
df


In [ ]:
# Set the positions and width for the bars
positions = range(len(df))
width = 0.35

# Plotting the bar chart
fig, ax1 = plt.subplots()

# Bars for Revenue and Profit
revenue_bars = ax1.bar(positions, df['Revenue (k$)']/100, width, label='Revenue (k$)', color='blue')
profit_bars = ax1.bar([p + width for p in positions], df['Profit (k$)']/100, width, label='Profit (k$)', color='orange')

# Add the second y-axis for the Profit Ratio
ax2 = ax1.twinx()
profit_ratio_dots, = ax2.plot([p + width/2 for p in positions], df['Profit Ratio'], 'ro', label='Profit Ratio')

# Set the ticks and ticklabels for the x-axis
ax1.set_xticks([p + width/2 for p in positions])
ax1.set_xticklabels(['S-1Seg', 'NS-1Seg', 'S-5Seg', 'NS-5Seg','S-1Seg', 'NS-1Seg', 'S-5Seg', 'NS-5Seg'])

# Set the axis labels
# ax1.set_xlabel('Segment')
ax1.set_ylabel('Revenue/Profit (k$)')
ax2.set_ylabel('Profit Ratio')

# Set the title of the graph
plt.title('Revenue, Profit and Profit Ratio (per MWh)')

# Create a legend by first getting handles and labels for the first y-axis and then the second y-axis
handles, labels = ax1.get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(handles + [profit_ratio_dots], labels + labels2, loc='upper right')

# Adjust layout to make room for the legend
plt.tight_layout()

# Show the plot
plt.show()

## Generator Type

In [74]:
Zone = 6

In [75]:
ThermalGen = pd.read_csv('2032 ADS PCM V2.4.1 Public Data/Processed Data/2022/ThermalGen_Full_C.csv')

In [76]:
ThermalGen['SubType'].unique()

array(['Bio-ICE', 'CT-NatGas-Aero', 'Bio-CT', 'ST-NatGas', 'ST-Other',
       'ICE-NatGas', 'Bio-ST', 'Geo-BinaryCycle', 'CT-NatGas-Industrial',
       'CCWhole-NatGas-SingleShaft', 'CT-OilDistillate', 'ST-Nuclear',
       'ST-WasteHeat', 'ST-Coal', 'CCWhole-NatGas-Aero', 'Bio-CC',
       'CCWhole-NatGas-Industrial', 'Geo-DoubleFlash',
       'ICE-OilDistillate', 'CT-Aero'], dtype=object)

In [77]:
ThermalGen['Fuel Name'].unique()

array(['Bio_Landfill_Gas', 'NG_Cal_PG&E LT', 'Petroleum Coke', 'Bio_Wood',
       'Geothermal', 'NG_Cal_PG&E BB', 'Oil_DistillateFuel_2',
       'Bio_Blk_Liquor', 'NG_Washington_Sumas', 'NG_Oregon_Malin',
       'NG_Idaho_Opal', 'Bio_Solid_Waste', 'NG_Oregon_Sumas', 'Uranium',
       'NG_West Texas_Waha', 'NG_Cal_Rosarito', 'NG_Cal_SDG&E',
       'NG_Cal_SoCalGas', 'Waste_Heat', 'NG_AZ/Cal_Blythe', 'NG_Cal_Kern',
       'Bio_Agri_Res', 'Coal_Jim_Bridger', 'NG_Wyoming',
       'NG_Utah_Opal Kern', 'NG_Montana', 'Coal_Centennial_Hard',
       'Coal_Colstrip', 'Coal_Bonanza', 'Coal_Dave_Johnston',
       'Coal_Hunter', 'Coal_Huntington', 'Coal_Sunnyside', 'Coal_Wyodak',
       'Coal_Laramie_River', 'NG_AZ_North-South', 'Coal_Naughton',
       'NG_New Mexico_South', 'NG_New Mexico_North', 'Coal_Coronado',
       'Coal_Springervile1-2', 'Coal_Springervile3-4', 'Coal_Apache',
       'NG_Nevada_South', 'NG_Nevada_North', 'Coal_Valmy',
       'NG_Colorado_Cheyenne', 'Coal_Pawnee', 'Coal_Rawhid

### Coal

In [78]:
ThermalGen[(ThermalGen['Zone']==Zone)&((ThermalGen['SubType']=='ST-Coal'))]

,Unnamed: 0,Generator Name,If Use Generic IO Curve,Generic IO Curve Name,IONumBlock,IOMaxCap(MW),IOMinCap(MW),MinInput(MMBTu),IncCap2(MW),IncHR2(MMBTu/MWh),...,Fuel Cost($/MMBTu),Fuel Name_Start,Fuel Cost($/MMBTu)_Start,Start Up Cost($),NoLoadCost($),IncCost2($/MW),IncCost3($/MW),IncCost4($/MW),IncCost5($/MW),IncCost6($/MW)
468,489,Coronado CO1,False,NaN,5,380.0,138.182,1857.4930,48.363,10.931,...,1.975636,Oil_DistillateFuel_2,24.566520,175150.636400,3669.730041,21.595677,22.050073,22.563739,23.247309,0.0
469,490,Coronado CO2,False,NaN,5,382.0,170.000,2077.7890,42.400,10.831,...,1.975636,Oil_DistillateFuel_2,24.566520,176072.478960,4104.954749,21.398114,21.842632,22.342468,23.010232,0.0
470,491,Springerville 1,False,NaN,4,387.0,210.000,1976.5030,35.400,8.657,...,1.314635,Oil_DistillateFuel_2,24.566520,178377.095360,2598.380021,11.380795,11.620059,12.003932,0.000000,0.0
471,492,Springerville 2,False,NaN,4,406.0,255.000,2309.1680,30.200,8.541,...,1.314635,Oil_DistillateFuel_2,24.566520,187134.629680,3035.713074,11.228298,11.417605,11.721286,0.000000,0.0
480,501,Springerville 4,False,NaN,4,415.0,264.091,2742.5820,30.182,9.309,...,1.707236,Oil_DistillateFuel_2,24.566520,191282.931200,4682.234723,15.892660,15.931926,15.995094,0.000000,0.0
492,513,Apache ST3,False,NaN,4,175.0,49.359,629.9562,25.128,10.314,...,2.060893,NG_AZ_North-South,3.485849,34558.430499,1298.272323,21.256050,22.342141,24.081535,0.000000,0.0
698,724,Springerville 3,False,NaN,4,417.0,271.753,2830.3220,29.049,9.375,...,1.707236,Oil_DistillateFuel_2,24.566520,192204.773760,4832.027610,16.005337,16.044604,16.106064,0.000000,0.0


In [79]:
ThermalGen[(ThermalGen['Zone']==Zone)&((ThermalGen['SubType']=='ST-Coal'))]['Fuel Name'].unique()

array(['Coal_Coronado', 'Coal_Springervile1-2', 'Coal_Springervile3-4',
       'Coal_Apache'], dtype=object)

In [80]:
ThermalGen[(ThermalGen['Zone']==Zone)&((ThermalGen['SubType']=='ST-Coal'))]['IOMaxCap(MW)'].sum()/1000

2.562

### ST

In [81]:
ThermalGen[(ThermalGen['Zone']==Zone)&((ThermalGen['SubType']=='ST-NatGas')|(ThermalGen['SubType']=='ST-Other')|(ThermalGen['SubType']=='ST-WasteHeat'))]

,Unnamed: 0,Generator Name,If Use Generic IO Curve,Generic IO Curve Name,IONumBlock,IOMaxCap(MW),IOMinCap(MW),MinInput(MMBTu),IncCap2(MW),IncHR2(MMBTu/MWh),...,Fuel Cost($/MMBTu),Fuel Name_Start,Fuel Cost($/MMBTu)_Start,Start Up Cost($),NoLoadCost($),IncCost2($/MW),IncCost3($/MW),IncCost4($/MW),IncCost5($/MW),IncCost6($/MW)
400,416,Reeves 1,False,NaN,3,44.0,10.00000,167.7857,11.33300,8.550,...,3.397497,NG_New Mexico_South,3.397497,4356.467524,570.051370,29.048597,36.543475,0.000000,0.0,0.0
401,417,Reeves 2,False,NaN,3,44.0,10.00000,154.5922,11.33300,9.514,...,3.397497,NG_New Mexico_South,3.397497,4356.467524,525.226497,32.323784,35.945516,0.000000,0.0,0.0
402,418,Reeves 3,False,NaN,3,66.0,15.00000,207.2389,17.00000,9.210,...,3.397497,NG_New Mexico_South,3.397497,4583.858653,704.093489,31.290945,36.183340,0.000000,0.0,0.0
454,475,Agua Fria 1,False,NaN,3,114.0,45.00000,469.6729,23.00000,8.860,...,3.485849,NG_AZ_North-South,3.485849,11327.156582,1637.208965,30.884625,32.763498,0.000000,0.0,0.0
455,476,Agua Fria 2,False,NaN,3,114.0,45.00000,474.5149,23.00000,8.951,...,3.485849,NG_AZ_North-South,3.485849,11327.156582,1654.087448,31.201837,33.101625,0.000000,0.0,0.0
456,477,Agua Fria 3,False,NaN,4,184.0,60.00000,645.1573,24.80000,9.245,...,3.485849,NG_AZ_North-South,3.485849,18213.274130,2248.921144,32.226677,32.589205,33.164371,0.0,0.0
474,495,H Wilson Sundt ST3,False,NaN,3,104.0,20.00000,255.9147,28.00000,9.356,...,3.485849,NG_AZ_North-South,3.485849,10478.867486,892.080086,32.613606,37.078979,0.000000,0.0,0.0
491,512,Apache ST2,False,NaN,4,175.0,50.00000,669.2489,25.00000,10.345,...,3.485849,NG_AZ_North-South,3.485849,34558.430499,2332.900832,36.061111,37.392706,39.522560,0.0,0.0
769,795,H Wilson Sundt 4 Gas,False,NaN,4,156.0,40.00000,472.6180,23.20000,9.023,...,3.485849,NG_AZ_North-South,3.485849,15568.599064,1647.475140,31.452819,33.248031,36.127342,0.0,0.0
1068,1156,Yuma Axis1ST86.7,False,NaN,3,86.7,28.89711,1218.9020,28.89971,5.392,...,3.628172,NG_BC_Sumas,3.628172,22826.580794,4422.386513,19.563105,21.134104,0.000000,0.0,0.0


In [82]:
ThermalGen[(ThermalGen['Zone']==Zone)&((ThermalGen['SubType']=='ST-NatGas')|(ThermalGen['SubType']=='ST-Other')|(ThermalGen['SubType']=='ST-WasteHeat'))]['IOMaxCap(MW)'].sum()/1000

1.0877000000000001

### CC

In [83]:
ThermalGen[(ThermalGen['Zone']==Zone)&((ThermalGen['SubType']=='CCWhole-NatGas-SingleShaft')|(ThermalGen['SubType']=='CCWhole-NatGas-Aero')|(ThermalGen['SubType']=='CCWhole-NatGas-Industrial')|(ThermalGen['SubType']=='CCPart-Steam'))]

,Unnamed: 0,Generator Name,If Use Generic IO Curve,Generic IO Curve Name,IONumBlock,IOMaxCap(MW),IOMinCap(MW),MinInput(MMBTu),IncCap2(MW),IncHR2(MMBTu/MWh),...,Fuel Cost($/MMBTu),Fuel Name_Start,Fuel Cost($/MMBTu)_Start,Start Up Cost($),NoLoadCost($),IncCost2($/MW),IncCost3($/MW),IncCost4($/MW),IncCost5($/MW),IncCost6($/MW)
316,318,HarquahalaCC1-Total,False,NaN,2,365.000000,203.280,1561.0180,161.720,5.411,...,3.485849,NG_AZ_North-South,3.485849,28324.157382,5441.473555,18.861931,0.000000,0.000000,0.000000,0.0
317,319,HarquahalaCC2-Total,False,NaN,2,365.000000,205.999,1535.2190,159.001,5.807,...,3.485849,NG_AZ_North-South,3.485849,28324.157382,5351.542128,20.242327,0.000000,0.000000,0.000000,0.0
318,320,HarquahalaCC3-Total,False,NaN,2,365.000000,207.561,1555.0910,157.439,4.974,...,3.485849,NG_AZ_North-South,3.485849,28324.157382,5420.812926,17.338615,0.000000,0.000000,0.000000,0.0
403,419,Afton CC,False,NaN,5,250.000000,150.000,1135.8590,23.333,4.920,...,3.397497,NG_New Mexico_South,3.397497,19546.266706,3859.077261,16.715684,21.003325,25.290966,33.974968,0.0
420,441,West Phoenix 1B,False,NaN,2,92.000000,50.600,509.1670,41.400,8.085,...,3.485849,NG_AZ_North-South,3.485849,7240.904365,1774.879448,28.183092,0.000000,0.000000,0.000000,0.0
421,442,West Phoenix 2B,False,NaN,2,92.000000,50.600,519.4780,41.400,8.248,...,3.485849,NG_AZ_North-South,3.485849,7240.904365,1810.822040,28.751285,0.000000,0.000000,0.000000,0.0
422,443,West Phoenix 3B,False,NaN,2,92.000000,50.600,507.2520,41.400,8.054,...,3.485849,NG_AZ_North-South,3.485849,7240.904365,1768.204046,28.075031,0.000000,0.000000,0.000000,0.0
425,446,WestPhoenix CC4,False,NaN,3,118.800000,77.220,676.9780,13.846,5.884,...,3.485849,NG_AZ_North-South,3.485849,9346.970536,2359.843310,20.510737,23.079808,0.000000,0.000000,0.0
462,483,Kyrene 7 CC,False,NaN,5,277.000000,156.000,1164.3340,21.333,5.592,...,3.485849,NG_AZ_North-South,3.485849,20377.650395,4058.692898,19.492869,20.461936,21.431002,29.629719,0.0
463,484,Santan 1,False,NaN,3,94.000000,61.100,561.1220,10.956,6.164,...,3.485849,NG_AZ_North-South,3.485849,7395.750761,1955.986750,21.486775,24.177851,0.000000,0.000000,0.0


In [84]:
ThermalGen[(ThermalGen['Zone']==Zone)&((ThermalGen['SubType']=='CCWhole-NatGas-SingleShaft')|(ThermalGen['SubType']=='CCWhole-NatGas-Aero')|(ThermalGen['SubType']=='CCWhole-NatGas-Industrial')|(ThermalGen['SubType']=='CCPart-Steam'))]['IOMaxCap(MW)'].sum()/1000

12.148940001000001

### CT

In [85]:
ThermalGen[(ThermalGen['Zone']==Zone)&((ThermalGen['SubType']=='CT-NatGas-Aero')|(ThermalGen['SubType']=='CT-NatGas-Industrial')|(ThermalGen['SubType']=='CT-OilDistillate')|(ThermalGen['SubType']=='CT-Aero')|(ThermalGen['SubType']=='CT-AB-Cogen'))]

,Unnamed: 0,Generator Name,If Use Generic IO Curve,Generic IO Curve Name,IONumBlock,IOMaxCap(MW),IOMinCap(MW),MinInput(MMBTu),IncCap2(MW),IncHR2(MMBTu/MWh),...,Fuel Cost($/MMBTu),Fuel Name_Start,Fuel Cost($/MMBTu)_Start,Start Up Cost($),NoLoadCost($),IncCost2($/MW),IncCost3($/MW),IncCost4($/MW),IncCost5($/MW),IncCost6($/MW)
109,109,Copper_1,False,NaN,3,68.0,10.0,348.1817,19.333,9.449,...,3.316763,NG_West Texas_Waha,3.316763,3836.673365,1154.836325,31.340098,36.640285,0.0,0.0,0.0
110,110,Montana_4_G4,False,NaN,3,97.0,50.0,502.4456,15.667,6.812,...,3.316763,NG_West Texas_Waha,3.316763,4174.750828,1666.493185,22.593792,24.262124,0.0,0.0,0.0
111,111,Montana_3_G3,False,NaN,3,97.0,50.0,516.5762,15.667,7.194,...,3.316763,NG_West Texas_Waha,3.316763,4174.750828,1713.361042,23.860796,24.427963,0.0,0.0,0.0
112,112,Montana_2_G2,False,NaN,3,97.0,50.0,506.7479,15.667,6.747,...,3.316763,NG_West Texas_Waha,3.316763,4144.792828,1680.762896,22.378203,23.310213,0.0,0.0,0.0
113,113,Montana_1_G1,False,NaN,3,93.2,50.0,501.2907,14.400,6.879,...,3.316763,NG_West Texas_Waha,3.316763,4144.792828,1662.662655,22.816016,23.940398,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,551,Coachella 2,False,NaN,2,20.0,10.0,166.7580,10.000,7.686,...,24.566520,Oil_DistillateFuel_2,24.566520,3406.319712,4096.663742,188.818273,0.000000,0.0,0.0,0.0
531,552,Coachella 3,False,NaN,2,20.0,10.0,166.7580,10.000,7.686,...,24.566520,Oil_DistillateFuel_2,24.566520,3406.319712,4096.663742,188.818273,0.000000,0.0,0.0,0.0
532,553,Coachella 4,False,NaN,2,20.0,10.0,166.7580,10.000,7.686,...,24.566520,Oil_DistillateFuel_2,24.566520,3406.319712,4096.663742,188.818273,0.000000,0.0,0.0,0.0
736,762,Douglas 1,False,NaN,2,16.0,8.0,151.4660,8.000,8.727,...,24.566520,Oil_DistillateFuel_2,24.566520,4858.124734,3720.992518,214.392020,0.000000,0.0,0.0,0.0


In [86]:
ThermalGen[(ThermalGen['Zone']==Zone)&((ThermalGen['SubType']=='CT-NatGas-Aero')|(ThermalGen['SubType']=='CT-NatGas-Industrial')|(ThermalGen['SubType']=='CT-OilDistillate')|(ThermalGen['SubType']=='CT-Aero')|(ThermalGen['SubType']=='CT-AB-Cogen'))]['IOMaxCap(MW)'].sum()/1000

4.041501

### Nuclear

In [87]:
ThermalGen[(ThermalGen['Zone']==Zone)&((ThermalGen['SubType']=='ST-Nuclear'))]

,Unnamed: 0,Generator Name,If Use Generic IO Curve,Generic IO Curve Name,IONumBlock,IOMaxCap(MW),IOMinCap(MW),MinInput(MMBTu),IncCap2(MW),IncHR2(MMBTu/MWh),...,Fuel Cost($/MMBTu),Fuel Name_Start,Fuel Cost($/MMBTu)_Start,Start Up Cost($),NoLoadCost($),IncCost2($/MW),IncCost3($/MW),IncCost4($/MW),IncCost5($/MW),IncCost6($/MW)
414,435,Palo Verde 1,False,NaN,1,1333.0,1333.0,14389.10,0.0,0.0,...,0.737556,Uranium,0.737556,170531.1,10612.772795,0.0,0.0,0.0,0.0,0.0
415,436,Palo Verde 2,False,NaN,1,1336.0,1336.0,14237.06,0.0,0.0,...,0.737556,Uranium,0.737556,170902.9,10500.634720,0.0,0.0,0.0,0.0,0.0
416,437,Palo Verde 3,False,NaN,1,1334.0,1334.0,14134.51,0.0,0.0,...,0.737556,Uranium,0.737556,170655.0,10424.998311,0.0,0.0,0.0,0.0,0.0


In [88]:
ThermalGen[(ThermalGen['Zone']==Zone)&((ThermalGen['SubType']=='ST-Nuclear'))]['IOMaxCap(MW)'].sum()/1000

4.003

### Geothermal

In [89]:
ThermalGen[(ThermalGen['Zone']==Zone)&((ThermalGen['SubType']=='Geo-BinaryCycle')|(ThermalGen['SubType']=='Geo-DoubleFlash'))]

,Unnamed: 0,Generator Name,If Use Generic IO Curve,Generic IO Curve Name,IONumBlock,IOMaxCap(MW),IOMinCap(MW),MinInput(MMBTu),IncCap2(MW),IncHR2(MMBTu/MWh),...,Fuel Cost($/MMBTu),Fuel Name_Start,Fuel Cost($/MMBTu)_Start,Start Up Cost($),NoLoadCost($),IncCost2($/MW),IncCost3($/MW),IncCost4($/MW),IncCost5($/MW),IncCost6($/MW)
517,538,Heber Geo Complex,False,NaN,2,52.00,26.000,259.5171,26.000,4.11,...,0.0,Geothermal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
519,540,Leathers,False,NaN,2,42.00,21.000,209.6100,21.000,4.11,...,0.0,Geothermal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
520,541,Del Ranch Company,False,NaN,2,42.00,21.000,209.6100,21.000,4.11,...,0.0,Geothermal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
522,543,JJ Elmore,False,NaN,2,42.00,21.000,209.6100,21.000,4.11,...,0.0,Geothermal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
523,544,Vulcan 1,False,NaN,2,38.00,19.000,189.6471,19.000,4.11,...,0.0,Geothermal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
524,545,Vulcan 2,False,NaN,2,10.74,5.370,53.6003,5.370,4.11,...,0.0,Geothermal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
525,546,Salton Sea 4,False,NaN,2,42.00,21.000,209.6100,21.000,4.11,...,0.0,Geothermal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
533,554,Second Imperial01-12,False,NaN,2,33.00,16.500,164.6936,16.500,4.11,...,0.0,Geothermal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
534,555,Hudson Ranch I,False,NaN,2,64.80,32.400,323.3983,32.400,4.11,...,0.0,Geothermal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
535,556,Salton Sea Unit 2 G2,False,NaN,2,5.00,2.500,24.9536,2.500,4.11,...,0.0,Geothermal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
ThermalGen[(ThermalGen['Zone']==Zone)&((ThermalGen['SubType']=='Geo-BinaryCycle')|(ThermalGen['SubType']=='Geo-DoubleFlash'))]['IOMaxCap(MW)'].sum()/1000

0.7788200000000001

## BIO & ICE

In [127]:
Zone = 6

In [107]:
ThermalGen[(ThermalGen['Zone']==Zone)&((ThermalGen['SubType']=='Bio-ICE')|(ThermalGen['SubType']=='Bio-CT')|(ThermalGen['SubType']=='Bio-ST')|(ThermalGen['SubType']=='Bio-CC'))]

,Unnamed: 0,Generator Name,If Use Generic IO Curve,Generic IO Curve Name,IONumBlock,IOMaxCap(MW),IOMinCap(MW),MinInput(MMBTu),IncCap2(MW),IncHR2(MMBTu/MWh),...,Fuel Cost($/MMBTu),Fuel Name_Start,Fuel Cost($/MMBTu)_Start,Start Up Cost($),NoLoadCost($),IncCost2($/MW),IncCost3($/MW),IncCost4($/MW),IncCost5($/MW),IncCost6($/MW)
141,141,Puente_Hills1,False,NaN,2,50.00,2.500,178.50000,47.500,11.900,...,2.378763,Bio_Landfill_Gas,2.378763,293.558615,424.609195,28.307280,0.000000,0.000000,0.0,0.0
143,143,SE_Resource,False,NaN,4,33.00,3.891,67.35200,8.979,13.637,...,0.000000,Bio_Solid_Waste,0.000000,185.898700,0.000000,0.000000,0.000000,0.000000,0.0,0.0
303,303,Brea2CC-Total,False,NaN,2,37.20,36.828,168.37800,0.372,7.167,...,2.378763,Bio_Landfill_Gas,2.378763,3332.263596,400.531356,17.048594,0.000000,0.000000,0.0,0.0
777,808,SANIGEN_24324_D1,False,NaN,4,8.00,2.400,93.48267,1.600,17.157,...,2.378763,Bio_Landfill_Gas,2.378763,624.635810,222.373117,40.812437,42.044636,42.047015,0.0,0.0
953,1017,ELLIS QFS,False,NaN,4,12.00,3.600,140.22400,2.400,23.157,...,2.378763,Bio_Landfill_Gas,2.378763,70.454066,333.559663,55.085015,56.317214,56.319593,0.0,0.0
973,1046,CHINO QFS,False,NaN,2,0.58,0.500,93.48267,0.080,17.157,...,2.378763,Bio_Landfill_Gas,2.378763,624.635810,222.373117,40.812437,0.000000,0.000000,0.0,0.0
974,1047,MM Tulare,False,NaN,2,1.50,1.000,93.48267,0.500,17.157,...,2.378763,Bio_Landfill_Gas,2.378763,624.635810,222.373117,40.812437,0.000000,0.000000,0.0,0.0
975,1048,GOLETA QFS,False,NaN,1,0.40,0.400,54.92107,0.000,0.000,...,2.378763,Bio_Landfill_Gas,2.378763,27.594509,130.644209,0.000000,0.000000,0.000000,0.0,0.0
976,1049,Tulare BioMAT Fuel Cell,False,NaN,2,2.80,1.400,54.92107,1.400,24.157,...,2.378763,Bio_Landfill_Gas,2.378763,27.594509,130.644209,57.463778,0.000000,0.000000,0.0,0.0


In [114]:
ThermalGen[(ThermalGen['Zone']==Zone)&((ThermalGen['SubType']=='Bio-ICE')|(ThermalGen['SubType']=='Bio-CT')|(ThermalGen['SubType']=='Bio-ST')|(ThermalGen['SubType']=='Bio-CC'))]['Fuel Name'].unique()

array(['Bio_Landfill_Gas'], dtype=object)

In [128]:
ThermalGen[(ThermalGen['Zone']==Zone)&((ThermalGen['SubType']=='Bio-ICE')|(ThermalGen['SubType']=='Bio-CT')|(ThermalGen['SubType']=='Bio-ST')|(ThermalGen['SubType']=='Bio-CC'))]['IOMaxCap(MW)'].sum()/1000

0.081

In [120]:
ThermalGen[(ThermalGen['Zone']==Zone)&((ThermalGen['SubType']=='ICE-NatGas')|(ThermalGen['SubType']=='ICE-OilDistillate'))]

,Unnamed: 0,Generator Name,If Use Generic IO Curve,Generic IO Curve Name,IONumBlock,IOMaxCap(MW),IOMinCap(MW),MinInput(MMBTu),IncCap2(MW),IncHR2(MMBTu/MWh),...,Fuel Cost($/MMBTu),Fuel Name_Start,Fuel Cost($/MMBTu)_Start,Start Up Cost($),NoLoadCost($),IncCost2($/MW),IncCost3($/MW),IncCost4($/MW),IncCost5($/MW),IncCost6($/MW)
93,93,BoulderPark1,False,NaN,2,4.10,0.82,15.783,3.28,9.752,...,3.682885,NG_Oregon_Sumas,3.682885,42.509983,58.126977,35.915496,0.000000,0.000000,0.0,0.0
94,94,BoulderPark2,False,NaN,2,4.10,0.82,15.783,3.28,9.752,...,3.682885,NG_Oregon_Sumas,3.682885,42.509983,58.126977,35.915496,0.000000,0.000000,0.0,0.0
95,95,BoulderPark3,False,NaN,2,4.10,0.82,15.783,3.28,9.752,...,3.682885,NG_Oregon_Sumas,3.682885,42.509983,58.126977,35.915496,0.000000,0.000000,0.0,0.0
96,96,BoulderPark4,False,NaN,2,4.10,0.82,15.783,3.28,9.752,...,3.682885,NG_Oregon_Sumas,3.682885,42.509983,58.126977,35.915496,0.000000,0.000000,0.0,0.0
97,97,BoulderPark5,False,NaN,2,4.10,0.82,15.783,3.28,9.752,...,3.682885,NG_Oregon_Sumas,3.682885,42.509983,58.126977,35.915496,0.000000,0.000000,0.0,0.0
98,98,BoulderPark6,False,NaN,2,4.10,0.82,15.783,3.28,9.752,...,3.682885,NG_Oregon_Sumas,3.682885,42.509983,58.126977,35.915496,0.000000,0.000000,0.0,0.0
302,302,BangorSubmarineBase2,False,NaN,4,10.00,3.00,50.994,2.00,11.819,...,24.566520,Oil_DistillateFuel_2,24.566520,124.566520,1252.745121,290.351700,305.165311,339.386474,0.0,0.0
376,384,PortWestwrd2-01,False,NaN,2,18.70,3.74,53.135,14.96,7.198,...,3.682885,NG_Oregon_Sumas,3.682885,193.886995,195.690103,26.509407,0.000000,0.000000,0.0,0.0
377,385,PortWestwrd2-02,False,NaN,2,18.70,3.74,53.135,14.96,7.198,...,3.682885,NG_Oregon_Sumas,3.682885,193.886995,195.690103,26.509407,0.000000,0.000000,0.0,0.0
378,386,PortWestwrd2-03,False,NaN,2,18.70,3.74,53.135,14.96,7.198,...,3.682885,NG_Oregon_Sumas,3.682885,193.886995,195.690103,26.509407,0.000000,0.000000,0.0,0.0


In [130]:
ThermalGen[(ThermalGen['Zone']==Zone)&((ThermalGen['SubType']=='ICE-NatGas')|(ThermalGen['SubType']=='ICE-OilDistillate'))]['Fuel Name'].unique()

array(['NG_AZ_North-South'], dtype=object)

In [129]:
ThermalGen[(ThermalGen['Zone']==Zone)&((ThermalGen['SubType']=='ICE-NatGas')|(ThermalGen['SubType']=='ICE-OilDistillate'))]['IOMaxCap(MW)'].sum()/1000

0.0032

### Renewables

In [91]:
HydroGen = pd.read_csv('2032 ADS PCM V2.4.1 Public Data/Processed Data/2022/HydroZoneCap_C.csv')

In [92]:
HydroGen['MaxCap(MW)']/1000

0     8.445408
1     1.541771
2     0.005850
3    32.843033
4     8.037265
5     0.629800
6     0.000000
Name: MaxCap(MW), dtype: float64

In [93]:
SolarGen = pd.read_csv('2032 ADS PCM V2.4.1 Public Data/Processed Data/2022/Solar_C.csv')

In [94]:
SolarGen['SubType'].unique()

array(['SolarPV-NonTracking', 'SolarPV-Tracking', 'SolarThermal-CSP6'],
      dtype=object)

In [95]:
SolarGen[(SolarGen['Zone']==Zone)&(SolarGen['SubType']=='SolarThermal-CSP6')]['Capacity(MW)'].sum()/1000

0.25

In [96]:
SolarGen[(SolarGen['Zone']==Zone)&(SolarGen['SubType']!='SolarThermal-CSP6')]['Capacity(MW)'].sum()/1000

2.8051079999999997

In [97]:
WindGen = pd.read_csv('2032 ADS PCM V2.4.1 Public Data/Processed Data/2022/Wind_C.csv')

In [98]:
WindGen[WindGen['Zone']==Zone]['Capacity(MW)'].sum()/1000

2.29954

### Storage

In [45]:
Storage = pd.read_csv('2032 ADS PCM V2.4.1 Public Data/Processed Data/2022/Storage_C_4hr_5GW_Strategic.csv')

In [46]:
Storage['SubType'].unique()

array(['PS-Hydro', 'PS-HydroRPS', 'Battery Storage'], dtype=object)

In [47]:
Storage[(Storage['Zone']==Zone)&((Storage['SubType']=='PS-Hydro')|(Storage['SubType']=='PS-HydroRPS'))]

,Unnamed: 0,Name,SubType,DevStatus,Area Name,MinCap(MW),MaxCap(MW),InitialDispatch(MW),Zone,Efficiency,Duration(h),MaxCap(MWh),Strategic
0,34,LakeHodges1,PS-Hydro,Existing,CISD,-20.0,20.0,20.0,3.0,0.8,12.0,240.0,0
1,35,LakeHodges2,PS-Hydro,Existing,CISD,-20.0,20.0,20.0,3.0,0.8,12.0,240.0,0


In [48]:
Storage[(Storage['Zone']==Zone)&((Storage['SubType']=='PS-Hydro')|(Storage['SubType']=='PS-HydroRPS'))]['MaxCap(MW)'].sum()/1000

0.04

In [49]:
Storage[(Storage['Zone']==Zone)&((Storage['SubType']=='Battery Storage'))]

,Unnamed: 0,Name,SubType,DevStatus,Area Name,MinCap(MW),MaxCap(MW),InitialDispatch(MW),Zone,Efficiency,Duration(h),MaxCap(MWh),Strategic
36,2934,Top Gun Energy Storage,Battery Storage,Existing,CISD,-84.00,84.00,0.0,3.0,0.9,4.0,336.0,0
37,2935,Fallbrook Energy Storage_2,Battery Storage,Existing,CISD,-87.50,87.50,0.0,3.0,0.9,4.0,350.0,0
38,2936,Fallbrook Energy Storage,Battery Storage,Existing,CISD,-87.50,87.50,0.0,3.0,0.9,4.0,350.0,0
39,2937,Gateway Energy Storage,Battery Storage,Existing,CISD,-630.00,630.00,0.0,3.0,0.9,4.0,2520.0,0
55,3485,Eastern BESS 11BA7.5,Battery Storage,Existing,CISD,-18.75,18.75,0.0,3.0,0.9,4.0,75.0,0
56,3491,Vista Energy Storage1BA40,Battery Storage,Existing,CISD,-100.00,100.00,0.0,3.0,0.9,4.0,400.0,0
68,3919,CISD BA Agg 1,Battery Storage,Existing,CISD,-7.50,7.50,0.0,3.0,0.9,4.0,30.0,0


In [50]:
Storage[(Storage['Zone']==Zone)&((Storage['SubType']=='Battery Storage'))]['MaxCap(MW)'].sum()/1000

1.01525

## Plot

In [ ]:
HistoryPrice = pd.read_csv('output/20220101-20221231 CAISO Real-time Price.csv')
HistoryPrice = HistoryPrice.drop_duplicates(subset=['Date', 'hub'])
Pivot_HistoryPrice = HistoryPrice.pivot(index='Date', columns='hub', values='price')

case1 = 'Strategic/UC25ED1_Strategic_true_Seg1_Load1.0_Fuel1.2_Error0.25_ratio0.1_MIP0.1_DARTDP'
RTPrice1 = pd.read_csv('output/'+case1+'/EDprice.csv',header=None)*12
case5 = 'Strategic/UC25ED1_Strategic_true_Seg5_Load1.0_Fuel1.2_Error0.25_ratio0.1_MIP0.1_DARTDP'
RTPrice5 = pd.read_csv('output/'+case5+'/EDprice.csv',header=None)*12
case1mar = 'Strategic/UC25ED1_Strategic_true_Seg1_Load1.0_Fuel1.2_Error0.25_Margin_MIP0.1_NSDAB'
RTPrice1mar = pd.read_csv('output/'+case1+'/EDprice.csv',header=None)*12
case5mar = 'Strategic/UC25ED1_Strategic_true_Seg5_Load1.0_Fuel1.2_Error0.25_Margin_MIP0.1_NSDAB'
RTPrice5mar = pd.read_csv('output/'+case5+'/EDprice.csv',header=None)*12
case3 = 'DecUpdate/UC25ED1_Strategic_false_Seg1_Load1.0_Fuel1.2_Error0.25_5GWBES_1yr_ESBid_Emergen'
RTPrice3 = pd.read_csv('output/'+case3+'/EDprice.csv',header=None)*12

# Create a 2x4 grid of subplots
# fig, axs = plt.subplots(2, 4, figsize=(16, 9))
fig, axs = plt.subplots(1, 2, figsize=(10, 6))

# Loop through each column and each subplot
for idx, (col, ax) in enumerate(zip([0,1], axs.flatten())):
    sorted_vals = np.sort(Pivot_HistoryPrice.iloc[:,col].values)[::-1]
    sorted_vals1 = np.sort(RTPrice1[col].values)[::-1]
    sorted_vals2 = np.sort(RTPrice5[col].values)[::-1]
    sorted_vals1mar = np.sort(RTPrice1mar[col].values)[::-1]
    sorted_vals2mar = np.sort(RTPrice5mar[col].values)[::-1]
    sorted_vals3 = np.sort(RTPrice3[col].values)[::-1]
    
    # Calculate the "reverse" cumulative distribution
    cdf = np.arange(1, len(sorted_vals) + 1) / float(len(sorted_vals))
    cdf1 = np.arange(1, len(sorted_vals1) + 1) / float(len(sorted_vals1))
    cdf2 = np.arange(1, len(sorted_vals2) + 1) / float(len(sorted_vals2))
    cdf1mar = np.arange(1, len(sorted_vals1mar) + 1) / float(len(sorted_vals1mar))
    cdf2mar = np.arange(1, len(sorted_vals2mar) + 1) / float(len(sorted_vals2mar))
    cdf3 = np.arange(1, len(sorted_vals3) + 1) / float(len(sorted_vals3))

    
    # Plot the CDF
    ax.plot(cdf,sorted_vals)
    ax.plot(cdf1,sorted_vals1)
    ax.plot(cdf2,sorted_vals2)
#     ax.plot(cdf1mar,sorted_vals1mar, linestyle='--')
#     ax.plot(cdf2mar,sorted_vals2mar, linestyle='--')
    ax.plot(cdf3,sorted_vals3)

    ax.set_xlim(0.00, 1)
    ax.set_ylim(-50, 500)
    ax.legend(['Historical','Seg1', 'Seg5', 'Base'],fontsize=20)
    ax.set_title(f"High Price CDF of Region {idx+1}",fontsize=20)
    ax.tick_params(axis='both', which='major', labelsize=20)
    ax.set_ylabel("Real-time Price ($/MWh)",fontsize=20)

plt.tight_layout()
plt.show()

In [ ]:
HistoryPrice = pd.read_csv('output/20220101-20221231 CAISO Real-time Price.csv')
HistoryPrice = HistoryPrice.drop_duplicates(subset=['Date', 'hub'])
Pivot_HistoryPrice = HistoryPrice.pivot(index='Date', columns='hub', values='price')

case1 = 'Strategic/MarginalCost/UC25ED1_Strategic_true_ratio1.0_Seg1_BAW0_MC10.0'
RTPrice1 = pd.read_csv('output/'+case1+'/EDprice.csv',header=None)*12
case2 = 'Strategic/MarginalCost/UC25ED1_Strategic_true_ratio1.0_Seg1_BAW0_MC20.0'
RTPrice2 = pd.read_csv('output/'+case2+'/EDprice.csv',header=None)*12
case3 = 'Strategic/MarginalCost/UC25ED1_Strategic_true_ratio1.0_Seg1_BAW0_MC30.0'
RTPrice3 = pd.read_csv('output/'+case3+'/EDprice.csv',header=None)*12
case4 = 'Strategic/MarginalCost/UC25ED1_Strategic_true_ratio1.0_Seg1_BAW0_MC40.0'
RTPrice4 = pd.read_csv('output/'+case4+'/EDprice.csv',header=None)*12
case5 = 'Strategic/MarginalCost/UC25ED1_Strategic_true_ratio1.0_Seg1_BAW0_MC50.0'
RTPrice5 = pd.read_csv('output/'+case5+'/EDprice.csv',header=None)*12

# Create a 2x4 grid of subplots
# fig, axs = plt.subplots(2, 4, figsize=(16, 9))
fig, axs = plt.subplots(1, 2, figsize=(10, 6))

# Loop through each column and each subplot
for idx, (col, ax) in enumerate(zip([0,1], axs.flatten())):
    sorted_vals = np.sort(Pivot_HistoryPrice.iloc[:,col].values)[::-1]
    sorted_vals1 = np.sort(RTPrice1[col].values)[::-1]
    sorted_vals2 = np.sort(RTPrice2[col].values)[::-1]
    sorted_vals3 = np.sort(RTPrice3[col].values)[::-1]
    sorted_vals4 = np.sort(RTPrice4[col].values)[::-1]
    sorted_vals5 = np.sort(RTPrice5[col].values)[::-1]

    
    # Calculate the "reverse" cumulative distribution
    cdf = np.arange(1, len(sorted_vals) + 1) / float(len(sorted_vals))
    cdf1 = np.arange(1, len(sorted_vals1) + 1) / float(len(sorted_vals1))
    cdf2 = np.arange(1, len(sorted_vals2) + 1) / float(len(sorted_vals2))
    cdf3 = np.arange(1, len(sorted_vals3) + 1) / float(len(sorted_vals3))
    cdf4 = np.arange(1, len(sorted_vals4) + 1) / float(len(sorted_vals4))
    cdf5 = np.arange(1, len(sorted_vals5) + 1) / float(len(sorted_vals5))
    
    # Plot the CDF
    ax.plot(cdf,sorted_vals)
    ax.plot(cdf1,sorted_vals1)
    ax.plot(cdf2,sorted_vals2)
    ax.plot(cdf3,sorted_vals3)
    ax.plot(cdf4,sorted_vals4)
    ax.plot(cdf5,sorted_vals5)

    ax.set_xlim(0.01, 0.5)
    ax.set_ylim(50, 500)
    ax.legend(['Historical','MC10', 'MC20', 'MC30', 'MC40', 'MC50'],fontsize=20)
    ax.set_title(f"High Price CDF of Region {idx+1}",fontsize=20)
    ax.tick_params(axis='both', which='major', labelsize=20)
    ax.set_ylabel("Real-time Price ($/MWh)",fontsize=20)

plt.tight_layout()
plt.show()

In [ ]:
HistoryPrice = pd.read_csv('output/20220101-20221231 CAISO Real-time Price.csv')
HistoryPrice = HistoryPrice.drop_duplicates(subset=['Date', 'hub'])
Pivot_HistoryPrice = HistoryPrice.pivot(index='Date', columns='hub', values='price')

case1 = 'Strategic/MarginalCost/UC25ED1_Strategic_true_ratio1.0_Seg1_BAW0_MC10.0'
RTPrice1 = pd.read_csv('output/'+case1+'/EDprice.csv',header=None)*12
case2 = 'Strategic/MarginalCost/UC25ED1_Strategic_true_ratio1.0_Seg1_BAW0_MC20.0'
RTPrice2 = pd.read_csv('output/'+case2+'/EDprice.csv',header=None)*12
case3 = 'Strategic/MarginalCost/UC25ED1_Strategic_true_ratio1.0_Seg1_BAW0_MC30.0'
RTPrice3 = pd.read_csv('output/'+case3+'/EDprice.csv',header=None)*12
case4 = 'Strategic/MarginalCost/UC25ED1_Strategic_true_ratio1.0_Seg1_BAW0_MC40.0'
RTPrice4 = pd.read_csv('output/'+case4+'/EDprice.csv',header=None)*12
case5 = 'Strategic/MarginalCost/UC25ED1_Strategic_true_ratio1.0_Seg1_BAW0_MC50.0'
RTPrice5 = pd.read_csv('output/'+case5+'/EDprice.csv',header=None)*12

# Create a 2x4 grid of subplots
# fig, axs = plt.subplots(2, 4, figsize=(16, 9))
fig, axs = plt.subplots(1, 2, figsize=(10, 6))

# Loop through each column and each subplot
for idx, (col, ax) in enumerate(zip([0,1], axs.flatten())):
    sorted_vals = np.sort(Pivot_HistoryPrice.iloc[:,col].values)[::-1]
    sorted_vals1 = np.sort(RTPrice1[col].values)[::-1]
    sorted_vals2 = np.sort(RTPrice2[col].values)[::-1]
    sorted_vals3 = np.sort(RTPrice3[col].values)[::-1]
    sorted_vals4 = np.sort(RTPrice4[col].values)[::-1]
    sorted_vals5 = np.sort(RTPrice5[col].values)[::-1]

    
    # Calculate the "reverse" cumulative distribution
    cdf = np.arange(1, len(sorted_vals) + 1) / float(len(sorted_vals))
    cdf1 = np.arange(1, len(sorted_vals1) + 1) / float(len(sorted_vals1))
    cdf2 = np.arange(1, len(sorted_vals2) + 1) / float(len(sorted_vals2))
    cdf3 = np.arange(1, len(sorted_vals3) + 1) / float(len(sorted_vals3))
    cdf4 = np.arange(1, len(sorted_vals4) + 1) / float(len(sorted_vals4))
    cdf5 = np.arange(1, len(sorted_vals5) + 1) / float(len(sorted_vals5))
    
    # Plot the CDF
    ax.plot(cdf,sorted_vals)
    ax.plot(cdf1,sorted_vals1)
    ax.plot(cdf2,sorted_vals2)
    ax.plot(cdf3,sorted_vals3)
    ax.plot(cdf4,sorted_vals4)
    ax.plot(cdf5,sorted_vals5)

    ax.set_xlim(0.7, 1.01)
    ax.set_ylim(-50, 200)
    ax.legend(['Historical','MC10', 'MC20', 'MC30', 'MC40', 'MC50'],fontsize=20)
    ax.set_title(f"Low Price CDF of Region {idx+1}",fontsize=20)
    ax.tick_params(axis='both', which='major', labelsize=20)
    ax.set_ylabel("Real-time Price ($/MWh)",fontsize=20)

plt.tight_layout()
plt.show()

In [ ]:
HistoryPrice = pd.read_csv('output/20220101-20221231 CAISO Real-time Price.csv')
HistoryPrice = HistoryPrice.drop_duplicates(subset=['Date', 'hub'])
Pivot_HistoryPrice = HistoryPrice.pivot(index='Date', columns='hub', values='price')

case1 = 'Strategic/UC25ED1_Strategic_true_Seg1_Load1.0_Fuel1.2_Error0.25_25MW_MIP0.1_NSDAB'
RTPrice1 = pd.read_csv('output/'+case1+'/EDprice.csv',header=None)*12
case5 = 'Strategic/UC25ED1_Strategic_true_Seg5_Load1.0_Fuel1.2_Error0.25_25MW_MIP0.1_NSDAB'
RTPrice5 = pd.read_csv('output/'+case5+'/EDprice.csv',header=None)*12
case1mar = 'Strategic/UC25ED1_Strategic_true_Seg1_Load1.0_Fuel1.2_Error0.25_Margin_MIP0.1_NSDAB'
RTPrice1mar = pd.read_csv('output/'+case1+'/EDprice.csv',header=None)*12
case5mar = 'Strategic/UC25ED1_Strategic_true_Seg5_Load1.0_Fuel1.2_Error0.25_Margin_MIP0.1_NSDAB'
RTPrice5mar = pd.read_csv('output/'+case5+'/EDprice.csv',header=None)*12
case3 = 'DecUpdate/UC25ED1_Strategic_false_Seg1_Load1.0_Fuel1.2_Error0.25_5GWBES_1yr_ESBid_Emergen'
RTPrice3 = pd.read_csv('output/'+case3+'/EDprice.csv',header=None)*12

# Create a 2x4 grid of subplots
# fig, axs = plt.subplots(2, 4, figsize=(16, 9))
fig, axs = plt.subplots(1, 2, figsize=(10, 6))

# Loop through each column and each subplot
for idx, (col, ax) in enumerate(zip([0,1], axs.flatten())):
    sorted_vals = np.sort(Pivot_HistoryPrice.iloc[:,col].values)[::-1]
    sorted_vals1 = np.sort(RTPrice1[col].values)[::-1]
    sorted_vals2 = np.sort(RTPrice5[col].values)[::-1]
    sorted_vals1mar = np.sort(RTPrice1mar[col].values)[::-1]
    sorted_vals2mar = np.sort(RTPrice5mar[col].values)[::-1]
    sorted_vals3 = np.sort(RTPrice3[col].values)[::-1]
    
    # Calculate the "reverse" cumulative distribution
    cdf = np.arange(1, len(sorted_vals) + 1) / float(len(sorted_vals))
    cdf1 = np.arange(1, len(sorted_vals1) + 1) / float(len(sorted_vals1))
    cdf2 = np.arange(1, len(sorted_vals2) + 1) / float(len(sorted_vals2))
    cdf1mar = np.arange(1, len(sorted_vals1mar) + 1) / float(len(sorted_vals1mar))
    cdf2mar = np.arange(1, len(sorted_vals2mar) + 1) / float(len(sorted_vals2mar))
    cdf3 = np.arange(1, len(sorted_vals3) + 1) / float(len(sorted_vals3))

    
    # Plot the CDF
    ax.plot(cdf,sorted_vals)
    ax.plot(cdf1,sorted_vals1)
    ax.plot(cdf2,sorted_vals2)
#     ax.plot(cdf1mar,sorted_vals1mar, linestyle='--')
#     ax.plot(cdf2mar,sorted_vals2mar, linestyle='--')
    ax.plot(cdf3,sorted_vals3)

    ax.set_xlim(0.7, 0.9)
    ax.set_ylim(10, 60)
    ax.legend(['Historical', 'Seg1', 'Seg5', 'Base'],fontsize=20)
    ax.set_title(f"Low Price CDF of Region {idx+1}",fontsize=20)
    ax.tick_params(axis='both', which='major', labelsize=20)
    ax.set_ylabel("Real-time Price ($/MWh)",fontsize=20)

plt.tight_layout()
plt.show()

In [ ]:
sorted_vals_S1 = np.sort(S1)[::-1]
sorted_vals_S5 = np.sort(S5)[::-1]
plt.plot(range(len(S1)), sorted_vals_S1, label='1Seg', color='blue', linewidth=2)
plt.plot(range(len(S5)), sorted_vals_S5, label='5Seg', color='red', linewidth=2)
plt.title('Power CDF')
plt.legend()

In [ ]:
ESOC1 = pd.read_csv('output/Strategic/UC25ED1_Strategic_true_Seg1_Load1.0_Fuel1.2_Error0.25_test/EDSOCini.csv',header=None)
ESOC5 = pd.read_csv('output/Strategic/UC25ED1_Strategic_true_Seg5_Load1.0_Fuel1.2_Error0.25_test/EDSOCini.csv',header=None)
SOC1 = ESOC1[45]
SOC5 = ESOC5[45]


sorted_vals_SOC1 = np.sort(SOC1)[::-1]
sorted_vals_SOC5 = np.sort(SOC5)[::-1]
plt.plot(range(len(S1)), sorted_vals_SOC1, label='1Seg', color='blue', linewidth=2)
plt.plot(range(len(S5)), sorted_vals_SOC5, label='5Seg', color='red', linewidth=2)
plt.title('SOC CDF')
plt.legend()

In [ ]:
# ESOC = pd.read_csv('output/Strategic/UC25ED1_Strategic_true_Seg5_Load1.0_Fuel1.2_Error0.25_test/EDSOCini.csv',header=None)
# plt.plot(range(len(ESOC)), ESOC[45], label='Series 1', color='blue', linewidth=2)
